# Build your own OpenAI Agent

With the [new OpenAI API](https://openai.com/blog/function-calling-and-other-api-updates) that supports function calling, it's never been easier to build your own agent!

In this notebook tutorial, we showcase how to write your own OpenAI agent in **under 50 lines of code**! It is minimal, yet feature complete (with ability to carry on a conversation and use tools).

## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the OpenAI API (using our own `llama_index` LLM class)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

In [1]:
import json
import os
from typing import Sequence, List

import openai
from dotenv import load_dotenv
from llama_index.llms import OpenAI, ChatMessage
from llama_index.tools import BaseTool, FunctionTool

import nest_asyncio

load_dotenv("../../../.env")
assert (key := os.getenv("OPENAI_API_KEY")); openai.api_key = key
nest_asyncio.apply()

Let's define some very simple calculator tools for our agent.

In [2]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [3]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

## Our (Slightly Better) `OpenAIAgent` Implementation 

We provide a (slightly better) `OpenAIAgent` implementation in LlamaIndex, which you can directly use as follows.  

In comparison to the simplified version above:
* it implements the `BaseChatEngine` and `BaseQueryEngine` interface, so you can more seamlessly use it in the LlamaIndex framework. 
* it supports multiple function calls per conversation turn
* it supports streaming
* it supports async endpoints
* it supports callback and tracing

In [4]:
from llama_index.agent import OpenAIAgent
from llama_index.agent.experimental_openai_agent import (
    ExperimentalOpenAIAgent, ChatStreamHandler
)
from llama_index.llms import OpenAI

In [5]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)
csh = ChatStreamHandler(llm)
exp_agent = ExperimentalOpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    stream_handler=csh,
    verbose=True
)

In [6]:
print(agent.chat("Tell me a random name"))

Sure! Here's a random name: "Elijah"


In [7]:
print(agent.chat("Now tell me that name backwards."))

The name "Elijah" spelled backwards is "hjaile".


In [8]:
response = exp_agent.chat("What is (121 * 3) + 42?")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405


### Chat

In [9]:
response = exp_agent.chat("What is (121 * 3) + 42?")
print(str(response))

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405
(121 * 3) + 42 is equal to 405.


In [10]:
# inspect sources
print(response.sources)

[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]


### Async Chat

In [11]:
response = await exp_agent.achat("What is 121 * 3?")
print(str(response))

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
121 multiplied by 3 is equal to 363.


### Streaming Chat
Here, every LLM response is returned as a generator. You can stream every incremental step, or only the last response.

In [12]:
response = agent.stream_chat(
    "What is 121 * 2? Once you have the answer, use that number to write a story about a group of mice."
)

response_gen = response.response_gen

for token in response_gen:
    print(token, end="")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 2
}
Got output: 242
The answer to 121 multiplied by 2 is 242.

Once upon a time, in a cozy little village, there lived a group of 242 mice. These mice were not your ordinary mice; they were adventurous and curious, always seeking new experiences and challenges.

One sunny morning, the leader of the mouse group, named Milo, gathered all the mice together and announced an exciting plan. He had heard rumors about a legendary cheese hidden deep within the nearby forest. The cheese was said to be the most delicious and flavorful cheese in the entire world.

With their hearts filled with excitement, the group of mice set off on their grand adventure. They navigated through tall grass, crossed babbling brooks, and climbed towering trees. Along the way, they encountered various obstacles, but their determination and teamwork helped them overcome every hurdle.

After days of tireless exploration, the mice finall

### Async Streaming Chat

In [13]:
response = await agent.astream_chat(
    "What is 121 + 8? Once you have the answer, use that number to write a story about a group of mice."
)

response_gen = response.response_gen

for token in response_gen:
    print(token, end="")

=== Calling Function ===
Calling function: add with args: {
  "a": 121,
  "b": 8
}
Got output: 129
The answer to 121 plus 8 is 129.

Once upon a time, in a peaceful meadow, there lived a group of 129 mice. These mice were known for their intelligence and resourcefulness, always finding innovative solutions to their everyday challenges.

One sunny morning, the leader of the mouse group, named Oliver, gathered all the mice together and shared an exciting plan. He had heard whispers of a magical cheese hidden deep within the enchanted forest. This cheese was said to possess extraordinary powers, granting anyone who tasted it with incredible abilities.

Filled with curiosity and anticipation, the group of mice embarked on a daring adventure into the mysterious forest. They navigated through thick foliage, crossed babbling streams, and climbed towering mushrooms. Along the way, they encountered various obstacles, but their unity and quick thinking helped them overcome each hurdle.

After da

### Agent with Personality

You can specify a system prompt to give the agent additional instruction or personality.

In [ ]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI
from llama_index.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
    system_prompt=SHAKESPEARE_WRITING_ASSISTANT,
)

In [ ]:
response = agent.chat("Hi")
print(response)

In [ ]:
response = agent.chat("Tell me a story")
print(response)